## 背景知识 

> ZooKeeper支持客户端读取和更新键值对d的操作具有高可用性。<br/ >
> 这种高可用性，是通过将数据复制存储在多个节点，并且允许客户端读取任意一个节点来实现的。<br/ >
> zookeeper实现这种设计的关键点在于，每个节点状态的变化，都是在之前状态的基础上所发生的增量改变，所有状态变化的顺序有隐藏的依赖关系。ZAB是驱动zookeeper保证这种顺序的协议，同时还负责了集群中leader主机的选举，恢复崩溃的leader主机节点等内容。以下是介绍ZAB协议的内容。<br/ >




## 基础定义
> * leader（决策者） 和 followers（追随者）-  在zookeeper集群中，一个节点拥有leader（决策者）角色，其他剩余节点拥有follower（追随者）角色。
>  leader（决策者）负责接收所有客户端发送的状态变更(写)请求，并且把这些状态变化复制给其他follower（追随者）和自身。<br/ >
> 所有的读请求都被均衡的分发到集群中所有的服务器，包括leader（决策者）和follower（追随者）。
> *  事务转发 – leader（决策者）把客户端状态的变更转发给其他所有follower（追随者）
> *‘e’ –leader(决策者)的时代号。每个服务器被分配到leader角色时，都会生成一个整数，这个整数必须比之前leader的时代号大。
> * ‘c’ – leader(决策者)生成的序列号，从0开始增加。和e（leader的时代号）一起，来对收到的客户端数据更新申请进行排序。
> * ‘F.history’ – follower（追随者）的历史操作队列。用于按照请求到达的2顺序，提交事务请求。
> * outstanding transactions – 未完成事务：- F.History中具有小于当前COMMIT序号的序列号的一组操作集合。


## ZAB 要求条件
> * Replication guarantees（事务转发保证）
>  1. 交付可靠性 - 如果一个服务器提交事务M，它将最终由所有服务器提交。
>   1. 完全顺序 - 如果事务A在事务B之前由一台服务器提交，则在所有服务器上，A都会在B之前被提交。 如果A和B是已经提交的消息，在所有服务器上，都保证a，b的提交顺序是一样的（A在B之前提交，或B在A之前提交）
>  1. 因果次序 - 如果一个服务器已经提交了事务A后，发送了事务B的请求，那么在所有服务器上，A一定会在B之前被执行。如果在发送B后发送C，那么C一定排在在B执行之后
> * 只要集群中大多数节（大于二分之一）点都处在正常工作状态，事务转发就会进行。
> * 当一个节点遇到故障，又重启之后，他应该补上故障期间所有的事务转发操作。


## ZAB 的实现方式
> * 客户端可以从任意zookeeper节点进行读操作。
> * 客户端任何改变了zookeeper节点的写操作，都会被转发给leader节点。
> * zookeeper使用2pc（两阶段提交）的一个变种，来给全部follower进行事务转发，follower接收到事务时，会将事务加入其F.history队列，并且向leader回复一个ACK，确认消息已收到。当leader收到集群的ACK时，便发出集群的commit请求；follower在收到commit请求时，会执行提交事务，除非c大于其F.history队列里存在的序列号（说明此更新申请事务之前有旧事务未完成），此时这个commit事务，会等待到队列中其他排序靠前的旧事务commit完成之后，再执行。
> * 如果leader主机崩溃，节点会执行恢复协议，在恢复正常操作之前，就共同的一致状态达成一致，并建立新的leader以广播状态改变。
> * 一个节点要获得leader角色，必须得到集群内规定数量节点的支持。由于节点可能崩溃，恢复，所以在一段时间内，可能存在多个leader，同一节点也可能多次拥有不同角色。

> *  节点的生命周期：每个节点一次执行一遍该协议的迭代，并且任何时候，先阶段迭代都可能被终止，并且开始一个从阶段0开始的新迭代。
>  + 阶段0——leader选举<br/ >
>  + 阶段1——发现阶段<br/ >
>     * 在这一阶段，followers与他们未来的的leader进行通信，以便leader接收到他的followers所进行的最近一次事务操作的信息。这一阶段的目的在于，发现集群中收到的最新更新请求的序列号，并且建立一个新的leader时代号，来废除前任leader。因为群组的follower把所有的更新请求都转发给前任leader，所以当前集群中，至少有一台follower，在其F.history’中，拥有前任leader接收到的所有更新请求。现任leader将同样拥有这些请求记录。阶段1的算法见[链接](https://distributedalgorithm.wordpress.com/2015/06/20/algorithm-for-zab-phase-1-discovery/?preview_id=156)。<br/ >
>     *  阶段 1和2对于集群之间相互一致的状态，尤其是从崩溃中恢复一致时，非常重要。
>  + 阶段2——同步阶段<br/ >
>     * 恢复一致协议阶段结束后，接下来是同步阶段。同步阶段主要是应用上一步leader获取的事务历史记录，来同步集群中其他服务器上的副本。leader会和follower通信，并且提供他的事务历史记录。如果他们follower副本确实比leader的记录要旧，会返回确认消息。当leader得到集群中的确认消息，会发送commit事务申请。此时是一个正式新leader被选举成功的标志。具体算法见[链接](https://distributedalgorithm.wordpress.com/2015/06/20/algorithm-for-zab-phase-2-discovery/?preview_id=163&preview_nonce=ecb4461ca8)。<br/ >

>  + 阶段3——广播阶段<br/ >
>     * 如果没有任何服务器崩溃，集群会永远处于这个阶段。一旦受到客户端的写请求，就会进行广播。具体算法见[链接](https://distributedalgorithm.wordpress.com/2015/06/20/algorithm-for-zab-phase-3-broadcast/?preview=true&preview_id=169&preview_nonce=318de41371)
>     * 为了检测失败，zab在leader和follower之间，采用了周期性的心跳检测。如果一个leader在一个超时时间内，没有接收到集群follower的心跳信息，则他放弃自己的leader身份，并且将集群状态转换到阶段0。一个follower若在超时时间内没有接收到leader的心跳信息，同时也会转到状态0.



